# MEDICAL CHATBOT

In [2]:
import os
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQAWithSourcesChain, create_retrieval_chain
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from llama_cpp import Llama
from langchain.llms import LlamaCpp

In [3]:
os.environ['PINECONE_API_KEY'] ='pcsk_5pg9q6_LDAcg5pvyr5htitdzjFdo6QaQDbbaJWWhs1XCPdVHwQPMcCW3hP7RdeEDztjsp9'

In [6]:
# Extract data from pdfs
def load_pdf (data):
    loader = DirectoryLoader(data,
                    glob='*.pdf',
                    loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents


In [7]:
extracted_data = load_pdf('../data/')

KeyboardInterrupt: 

In [ ]:
extracted_data[0]

Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:36:01-05:00', 'moddate': '2004-12-18T17:38:34-06:00', 'source': '../data/Gale Encyclopedia of Medicine Vol. 3 (G-M).pdf', 'total_pages': 941, 'page': 0, 'page_label': '1'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION')

In [ ]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 500,chunk_overlap = 20
    )
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [ ]:
text_chunks = text_split(extracted_data)
len(text_chunks)

37368

In [4]:
# Download embedding modle
def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [5]:
embeddings = download_hugging_face_embedding()

In [6]:
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [7]:
from pinecone import Pinecone

# Initialize the Pinecone client
pinecone_client = Pinecone(api_key="pcsk_5pg9q6_LDAcg5pvyr5htitdzjFdo6QaQDbbaJWWhs1XCPdVHwQPMcCW3hP7RdeEDztjsp9")


In [8]:
vectorstore = PineconeVectorStore.from_existing_index(
    index_name='medical-chatbot', 
    embedding=embeddings
    )

In [8]:
index_name = "medical-chatbot"

docsearch = PineconeVectorStore.from_texts(
    [t.page_content for t in text_chunks]
    ,embeddings,
    index_name=index_name,
                            )



NameError: name 'text_chunks' is not defined

In [9]:
vectorstore

In [10]:
prompt_template = """
Use the following pices of information to answer the user's questioin>
If you don't know the answer, just say that you don't konw, don't try to make up an anser.

Context: {context}
Question:{input}

Only return the helpful answer below and nothing else.
Helpful answer:

"""

In [11]:
prompt = PromptTemplate(
    template=prompt_template,input_variables=['context','input'],
)
chain_type_kwargs = {'prompt':prompt}

In [12]:
llm = LlamaCpp(model_path='../model/llama-2-7b-chat.ggmlv3.q4_0.bin',)
                   # model_type = 'llama',
                    #config={'max_new_tokens':512,
                         #   'temperature':0.8})

llama.cpp: loading model from ../model/llama-2-7b-chat.ggmlv3.q4_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_head_kv  = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 5.0e-06
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: mem required  = 3615.73 MB (+  256.00 MB per state)
llama_new_context_with_model: kv self size  =  256.00 MB
AVX =

In [5]:
print(os.path.getsize(model_path))  # Expected: ~5-7GB for 8B Q4_K_M

4920739232


In [1]:
from langchain_community.llms import LlamaCpp
llm = LlamaCpp(model_path='../model/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf',)
                   # model_type = 'llama',
                    #config={'max_new_tokens':512,
                    
                         #   'temperature':0.8})

llama.cpp: loading model from ../model/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf
error loading model: unknown (magic, version) combination: 46554747, 00000003; is this really a GGML file?
llama_load_model_from_file: failed to load model


ValidationError: 1 validation error for LlamaCpp
  Value error, Could not load Llama model from path: ../model/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf. Received error  [type=value_error, input_value={'model_path': '../model/...: None, 'grammar': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [7]:

!file ../model/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf

../model/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf: data


In [1]:
from llama_cpp import Llama
model_path='../model/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf'
llm = Llama(
    model_path=model_path,
    n_ctx=2048,  # Context window
    n_gpu_layers=40,  # Set to 0 if CPU-only
    verbose=True,
)
print(llm.create_completion("Hello!", max_tokens=512))

llama.cpp: loading model from ../model/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf
error loading model: unknown (magic, version) combination: 46554747, 00000003; is this really a GGML file?
llama_load_model_from_file: failed to load model


AssertionError: 

In [13]:
qa = create_retrieval_chain(
    retriever = vectorstore.as_retriever(search_kwargs={'k':2}),
    combine_docs_chain = create_stuff_documents_chain(llm = llm,prompt = prompt),

)

In [ ]:

user_input= input(f"Input Prompt:")
result = qa.invoke({"input":user_input})
print("Question :",result['input'])
print("Result : ",result['answer'])

Result :  Acne is a skin disorder in which the sebaceous glands become inflamed.



llama_print_timings:        load time =  3810.47 ms
llama_print_timings:      sample time =     7.29 ms /    21 runs   (    0.35 ms per token,  2879.47 tokens per second)
llama_print_timings: prompt eval time = 89363.06 ms /   271 tokens (  329.75 ms per token,     3.03 tokens per second)
llama_print_timings:        eval time =  9933.09 ms /    20 runs   (  496.65 ms per token,     2.01 tokens per second)
llama_print_timings:       total time = 99598.80 ms


In [15]:
result

{'input': 'what is acne?',
 'context': [Document(id='e295c9c4-3ce6-44d7-9e0f-a0cf0cce08b5', metadata={}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
  Document(id='330b134e-6c77-4fd9-991d-8f79ad056305', metadata={}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25')],
 'answer': 'Acne is a skin disorder in which the sebaceous glands become inflamed.'}